<h2><b><i>Radial Basis Function Neural Network</i></b></h2>

Σκοπός της παρούσας άσκησης είναι η κατηγοριοποίηση χειρόγραφων ψηφίων από το [MNIST dataset](https://www.wikiwand.com/en/MNIST_database) με τη χρήση νευρωνικών δικτύων [RBF](https://en.wikipedia.org/wiki/Radial_basis_function) (RBF). Για την κατηγοροποίηση των χειρόγραφων ψηφίων, χρησιμοποιείται ο αλγόριθμος K-means. Στη κλασική υλοποίηση του K-means όπου έχουμε το κέντρο του cluster και ταξινομομούμε κάθε σημείο με βάση την απόσταση από τα κέντρα. Σε αυτή την περίπτωση μπορεί να υπάρξουν σημεία που να μην μπορούν να αντιστοιχηθούν σε κάποιο cluster. Με τη χρήση του νευρωνικού δικτύου RBF κάθε σημείο ταξινομείται σε ομάδες με συγκεκριμένο κέντρο, γνωρίζοντας όμως τις αποστάσεις μεταξύ των κέντρων αλλά και με διαστήματα εμπιστοσύνης ως προς την πρόβλεψή μας. Με αυτό το τρόπο όλα τα σημεία αντιστοιχίζονται σε ένα cluster. Για να είναι ομαλή η ταξινόμηση των στοιχείων στα clusters μπορεί να χρησιμοποιηθεί μια εκθετική συνάρτηση υψωμένη στην αρνητική τιμή της απόστασης ως γινόμενο με έναν βαθμωτό συντελεστή beta, όπως φαίνεται παρακάτω:

![alt text](https://miro.medium.com/max/145/1*MIay3aIlpT18yewOfnvTiQ.png)

Στα πλαίσια του παραδείγματος για το β χρησιμοποιήθηκε ο παρακάτω τύπος

![alt text](https://miro.medium.com/max/608/1*YEcI_P6orY917fQrzHQEjQ.png)

Όπου k ο αριθμός των clusters και Dmax η μέγιστη ευκλείδια απόσταση μεταξύ των κέντρων. Το dataset που θα χρησιμοποιήσετε είναι διαθέσιμο [εδώ](https://drive.google.com/file/d/1WrVPB3C1vIApWNz5BaSuNmHFopf5htCg/view?usp=sharing) (θα πρέπει να το φορτώσετε στο Colab για να το τρέξετε). 

*Οι ερωτήσεις της άσκησης είναι μαζί με το αντίστοιχο βήμα στον κώδικα*


**Βήμα 1ο**

Τρέχετε τον κώδικα στον οποίο και ορίζονται οι συναρτήσεις που θα μας χρειαστούν. Στα πλαίσια της άσκησης χρησιμοποιούμε μία παραμετροποιημένη έκδοση της συνάρτησης K-Means, όπου μας επιστρέφει τα κέντρα των ομάδων (clusters) καθώς και την τυπική απόκλιση τους.

In [10]:
"""''

Copyright © 2020 Tarlan Ahadli

Permission is hereby granted, free of charge, to any person obtaining a
copy of this software and associated documentation files (the “Software”),
to deal in the Software without restriction, including without limitation
the rights to use, copy, modify, merge, publish, distribute, sublicense,
and/or sell copies of the Software, and to permit persons to whom the Software
is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included
in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY
KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE
WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE
AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
OTHER DEALINGS IN THE SOFTWARE.

''"""

import numpy as np


def get_distance(x1, x2):
    sum = 0
    for i in range(len(x1)):
        sum += (x1[i] - x2[i]) ** 2
    return np.sqrt(sum)


def kmeans(X, k, max_iters):
    centroids = X[np.random.choice(range(len(X)), k, replace=False)]
    # centroids = [np.random.uniform(size=len(X[0])) for i in range(k)]

    converged = False
    current_iter = 0

    while (not converged) and (current_iter < max_iters):

        cluster_list = [[] for i in range(len(centroids))]

        for x in X:  # Go through each data point
            distances_list = []
            for c in centroids:
                distances_list.append(get_distance(c, x))
            cluster_list[int(np.argmin(distances_list))].append(x)

        cluster_list = list((filter(None, cluster_list)))

        prev_centroids = centroids.copy()

        centroids = []

        for j in range(len(cluster_list)):
            centroids.append(np.mean(cluster_list[j], axis=0))

        pattern = np.abs(np.sum(prev_centroids) - np.sum(centroids))

        print('K-MEANS: ', int(pattern))

        converged = (pattern == 0)

        current_iter += 1

    return np.array(centroids), [np.std(x) for x in cluster_list]


**Βήμα 2ο**

Τρέχουμε την κλάση για το RBFNN

---

***Ερώτηση 1***

Να αναφέρετε επιγραμματικά με ποιον τρόπο λειτουργεί ένα νευρωνικό δίκτυο RBF.

***Απάντηση 1***


Tα συγκεκριμένα νευρωνικά δίκτυα, αποτελούνται από 3 στρώματα, 1 στρώμα εισόδου, 1 κρυφό επίπεδο και τέλος το επίπεδο εξόδου. Ουσιαστικά, το σημείο που διαφέρει από το απλό νευρωνικό δίκτυο εμπρόσθιας τροφοδότησης, είναι πως στο κρυφό επίπεδο αναλαμβάνει να εφαρμόσει ένα non-linear transformation στα inputs και να περάσει αυτά τα μετασχηματισμένα διανύσματα στο στρώμα εξόδου, το οποίο γίινεται με linear transformation. Βασικό στοιχείο στα RBFN είναι οι αποστάσεις των δεδομένων εισόδου από τα διανύσματα κέντρων που λέγονται είτε prototypes είτε centroids, που προκύπτει από το στάδιο του training. O μετασχηματισμός των δεδομένων αποτελείται από την radial basis function η οποία κάνει mapping δεδομένα από ένα χώρο d διαστάσεων σε ένα χώρο m διαστάσεων συνήθως m <= d. Γενικά, όσο μεγαλύτερο m τόσο μεγαλύτερη πιθανότητα να υπάρχει γραμμική διαχωρισιμότητα σύμφωνα με το θεώρημα του  cover.

Αν ονομάσουμε τη συνάρτηση rbf ως Φ : $R^{d} -> R^{m}$ Tότε σε κάθε κόμβο του κρυφού επιπέδου j αντιστοιχίζεται μια $Φ_{j}$, η οποία με τη σειρά της αντιστοιχίζεται και αυτή με έναν υποδοχέα ή αλλιώς receptor $t_{j}$, από τον οποίο υπολογίζονται οι αποστάσεις των δειγμάτων. H rbf Φ oυσιαστικά υπολογίζει την απόσταση κάθε δείγματος από αυτόν τον receptor, ο οποίος μπορεί να θεωρηθεί όπως είπαμε και προηγουμένως ως centroid ή prototype.

H πιο σύνηθες rbf είναι η γκαουσιανή η οποία έχει τη μορφή που φαίνεται στην αρχή της εκφώνησης $Φ(r) = e^{-β{r^2}}$ όπου $r = ||x-t||$ που συμβολίζει την απόσταση του δείγματος από το prototype ή centroid, ενώ $β$ συμβολίζει τη διασπορά όπως το variance σε μια gaussian distribution.

Eπομένως, από τα παραπάνω καταλαβαίνουμε πως το training των RBF ουσιαστικά απαιτεί την εκμάθηση των παραμέτρων t,β και των βαρών W. Αυτή η εκμάθηση μπορεί να γίνει με hybrid τρόπο. Tέλος παρουσιάζουμε μια εικόνα ενός RBF για να δείξουμε και γραφικά, ότι περιγράφηκε ως τώρα.

![alt text](
https://www.researchgate.net/publication/276915084/figure/fig3/AS:669044441096210@1536524033799/Architecture-of-RBF-network.png)

***Ερώτηση 2***

Στα πλαίσια του παραδείγματος, για κάθε αριθμό (κλάση) έχουμε μία μόνο ομάδα. Τι θα συνέβαινε αν είχαμε παραπάνω από μία ομάδες που θα αντιστοιχούσαν σε μία κλάση; Πώς λύνεται αυτό μέσω του RBF;

**Απάντηση 2**

Εφόσον ανά κλάση έχουμε μια μόνο ομάδα τότε απλά θα μπορούσαμε να δούμε τις τιμές των rbf για κάθε δείγμα, και η ομάδα με την μικρότερη απόσταση θα μας δώσει την κλάση που ανήκει το συγκεκριμένο δείγμα. Από την άλλη, αν είχαμε παραπάνω ομάδες για κάθε κλάση, τότε θα χρησιμοποιούσαμε και το 3ο επίπεδο, αυτό της εξόδου, για να κάνουμε και τους επιπλέον υπολογισμούς και να αξιοποιηθούν και οι γραμμικοί μετασχηματισμοί του output layer($Σw_jΦ_j(x)$ σύμφωνα με το παραπάνω σχήμα).





In [17]:
class RBF:

    def __init__(self, X, y, tX, ty, num_of_classes,
                 k, std_from_clusters=True):
        self.X = X
        self.y = y

        self.tX = tX
        self.ty = ty

        self.number_of_classes = num_of_classes
        self.k = k
        self.std_from_clusters = std_from_clusters

    def convert_to_one_hot(self, x, num_of_classes):
        arr = np.zeros((len(x), num_of_classes))
        for i in range(len(x)):
            c = int(x[i])
            arr[i][c] = 1
        return arr

    def get_rbf(self, x, c, s):
        distance = get_distance(x, c)
        return 1 / np.exp(-distance / s ** 2)

    def get_rbf_as_list(self, X, centroids, std_list):
        RBF_list = []
        for x in X:
            RBF_list.append([self.get_rbf(x, c, s) for (c, s) in zip(centroids, std_list)])
        return np.array(RBF_list)
    
    def fit(self):

        self.centroids, self.std_list = kmeans(self.X, self.k, 1000)

        if not self.std_from_clusters:
            dMax = np.max([get_distance(c1, c2) for c1 in self.centroids for c2 in self.centroids])
            self.std_list = np.repeat(dMax / np.sqrt(2 * self.k), self.k)

        RBF_X = self.get_rbf_as_list(self.X, self.centroids, self.std_list)

        self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.convert_to_one_hot(self.y, self.number_of_classes)

        RBF_list_tst = self.get_rbf_as_list(self.tX, self.centroids, self.std_list)

        self.pred_ty = RBF_list_tst @ self.w

        self.pred_ty = np.array([np.argmax(x) for x in self.pred_ty])

        diff = self.pred_ty - self.ty

        print('Accuracy: ', len(np.where(diff == 0)[0]) / len(diff))

**Βήμα 3ο**

Τρέχουμε τη συνάρτηση όπου το RBFNN γίνεται fit. Αρχικά τρέχει ο αλγόριθμος k-means για να βρούμε τα κέντρα και την τυπική απόκλιση των clusters. Στη συνέχεια, μπορούμε να χρησιμοποιήσουμε το ίδιο beta για όλα τα clusters


---
***Ερώτηση 3***

Με ποιούς τρόπους μπορούμε να υπολογίσουμε τα βάρη; Ποιες διαφορές υπάρχουν στους τρόπους αυτούς; Ποιος παρέχει τη καλύτερη δυνατή λύση;

***Απάντηση 3***

Γενικά, τα RBF μπορούν να εκπαιδευτούν με hybrid τρόπο. Δηλαδή να εκπαιδέυσουμε το 1ο layer με τα centroids με unsupervised τρόπο **π.χ. k-means για centroids και k-nn για το spread (διασπορά) του kernel** και για το επόμενο layer της εξόδου να εφαρμόσουμε τα παρακάτω:


Tα βάρη θα μπορούσαν να εκπαιδευτούν με τη χρήση του αλγορίθμου back propagation κάνοντας χρήση μια συνάρτηση κόστους όπως cross-entropy που θα ταίριαζε λόγω του ότι έχουμε πρόβλημα classification είτε άλλες φορές με MSE. Αρχικά θεωρούμε πως έχουμε Ν inputs για τα οποία ισχύει ότι ανήκουν στον $R^d$. Έχουμε $c$ centroids στο hidden layer και $m = 10$ νευρώνες στο επίπεδο εξόδου ένα για κάθε κλάση για το συγκεκριμένο πρόβλημα. Το back propagation μπορεί να γίνει με ποικίλους τρόπους gradient descent.
  

Παρακάτω περιγράφουμε έναν 2ο τρόπο ο οποίος υπάρχει στις διαφάνειες της μηχανικής μάθησης του κ. Σταφυλοπάτη.
Θεωρούμε για κάθε δείγμα $x_i$ ένα διάνυσμα $ h_i = (Φ_1(x_i), ..., Φ_c(x_i))$ που συμβολίζει τα activations στο hidden layer των rbf. Ως αποτέλεσμα έχουμε τον πίνακα $Η$ διαστάσεων $Νxc$, όπου κάθε γραμμή αντιστοιχεί σε ένα διάνυσμα $h_i$.
Τώρα, έχουμε $m=10$ νευρώνες στο στρώμα εξόδου, έναν για κάθε κλάση, και 10 διανύσματα βαρών $w_1, ..., w_{10}$ διάστασης c, ένα για κάθε νευρώνα. Ορίζουμε 10 διανύσματα $b_m στο \{0,1\} ^{10}$ που ορίζονται ως εξής: $b_{mi} = 1$, αν το $x_i$ ανήκει στην κλάση m, αλλιώς 0. 
  
Έχοντας ορίσει με αυτόν τον τρόπο τα διανύσματα b, μπορούμε να υπολογίσουμε τα βάρη w ως: $w_m = H^{+}b_m$  

όπου $H_{+}$ o ψευδοαντίστροφος του Η που ορίζειται ως $Η^{+} = (Η^{\top}Η)^{-1}Η^{\top}$ κι έχει διάσταση cxN


Ο 2ος τρόπος είναι πιο ακριβός υπολογιστικά από τον πρώτο καθώς έχει δύσκολες αλγεβρικές πράξεις. Σε περίπτωση μεγάλων πινάκων δηλαδή δεν είναι αποδοτικός. Επίσης, μπορεί να οδηγήσει και σε overfit των δεδομένων. O 1oς τρόπος είναι πιο γρήγορος και αποδοτικός και συνηθίζεται στην εκπαίδευση νευρωνικών δικτύων.

**Βήμα 4ο**

Σπάμε το dataset του MNIST σε training και testing και αφήνουμε το RBFNN να βγάλει το τελικό αποτέλεσμα.

---

***Ερώτηση 4***

Επιχειρείστε να αλλάξετε την τιμή του k σε 500. Τι παρατηρείτε ως προς την εκτέλεση;

***Απάντηση 4***

Αλλάζοντας την τιμή σε 500, παρατηρούμε πως ανεβαίνει η ακρίβεια του αλγορίθμου. Αυτό δικαιολογείται καθώς όσο πιο πολλά centroids έχουμε στο hidden layer τόσο πιο γραμμικά διαχωρίσιμες γίνονται οι κλάσεις μας, λόγω του θεωρήματος του cover. Ωστόσο αυτό αυξάνει το υπολογιστικό κόστος αρκετά, αφού αργεί να γίνει η σύγκλιση έχοντας παραπάνω διαστάσεις. (Παρακάτω φαίνεται το keyboard interrupt επειδή έπαιρνε πολύ ώρα στον υπολογιστή μου.)


In [18]:
##################################

data = np.load('mnist_data.npy').astype(float)

train_y = data[0:2500, 0]
train_x = data[0:2500, 1:]

test_y = data[0:300, 0]
test_x = data[0:300, 1:]

RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, num_of_classes=10,
                     k=10, std_from_clusters=False)

RBF_CLASSIFIER.fit()

K-MEANS:  59508
K-MEANS:  4968
K-MEANS:  2327
K-MEANS:  3109
K-MEANS:  785
K-MEANS:  1050
K-MEANS:  409
K-MEANS:  503
K-MEANS:  314
K-MEANS:  153
K-MEANS:  463
K-MEANS:  34
K-MEANS:  40
K-MEANS:  18
K-MEANS:  93
K-MEANS:  282
K-MEANS:  297
K-MEANS:  258
K-MEANS:  327
K-MEANS:  450
K-MEANS:  221
K-MEANS:  212
K-MEANS:  219
K-MEANS:  7
K-MEANS:  56
K-MEANS:  191
K-MEANS:  58
K-MEANS:  166
K-MEANS:  157
K-MEANS:  17
K-MEANS:  0
Accuracy:  0.63


In [19]:
##################################

data = np.load('mnist_data.npy').astype(float)

train_y = data[0:2500, 0]
train_x = data[0:2500, 1:]

test_y = data[0:300, 0]
test_x = data[0:300, 1:]

RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, num_of_classes=10,
                     k=500, std_from_clusters=False)

RBF_CLASSIFIER.fit()

KeyboardInterrupt: 